# LangSmith Tutorial (No LangChain)

Here, we'll do a very basic walkthrough of how you can get started with LangSmith in TypeScript without LangChain.

First, we'll do some setup. Create a LangSmith API Key by navigating to the settings page in LangSmith, then create an .env file with values for the following variables, in the same directory as this notebook:
```
OPENAI_API_KEY=<YOUR OPENAI API KEY>
LANGCHAIN_TRACING_V2=true
LANGCHAIN_PROJECT='langsmith-wikirag-walkthrough'
LANGCHAIN_API_KEY=<YOUR LANGSMITH API KEY>
```

In [1]:
import "dotenv/config"; // Load env vars from .env file
import OpenAI from "npm:openai";
import wiki from "npm:wikipedia";
import { Client } from "langsmith";
import { traceable } from "langsmith/traceable";

const openai = new OpenAI();
const langsmith = new Client();

[Module: null prototype] { default: {} }

In [2]:
// Wrap the OAI client in traceable to enable logging to LangSmith
const traceCreateCompletion = traceable(
  openai.chat.completions.create.bind(openai.chat.completions),
  { name: "OpenAI Chat Completion", run_type: "llm" }
);

## Create a Wikipedia Rag Pipeline that doesn't use LangChain

Here, we'll create a very simple RAG pipeline that:
1. Generates a Wikpedia search query from the input question
2. Retrieves relevant page summaries from Wikipedia based on the search query
3. Answers the input question based on the context from the retrieval step

In [3]:
async function generateWikiSearch(input: {question: string}): Promise<string> {
    const messages = [
        { role: "system", content: "Generate a search query to pass into wikipedia to answer the user's question. Return only the search query and nothing more. This will be passed in directly to the Wikipedia search engine." },
        { role: "user", content: input.question }
    ];

    const chatCompletion = await traceCreateCompletion({
        model: "gpt-3.5-turbo",
        messages: messages,
        temperature: 0,
    });
    return chatCompletion.choices[0].message.content;
}

In [4]:
function convertDocs(results: Array<{ summary: string; url: string }>): Array<{ page_content: string; type: string; metadata: { url: string } }> {
    // Convert docs to a format that LangSmith accepts (for nicer rendering)
    return results.map(r => ({
        page_content: r.summary,
        type: "Document",
        metadata: { url: r.url }
    }));
}

async function retrieve(input: {query: string, numDocuments: number}): Promise<Array<{ summary: string; url: string }>> {
    const { results } = await wiki.search(input.query, { limit: 10 });
    const finalResults: Array<{ summary: string; url: string }> = [];

    for (const result of results) {
        if (finalResults.length >= input.numDocuments) {
            // Just return the top 2 pages for now
            break;
        }
        const page = await wiki.page(result.title, { autoSuggest: false });
        const summary = await page.summary();
        finalResults.push({
            summary: summary.extract,
            url: page.fullurl
        });
    }
    
    return convertDocs(finalResults);
}

In [5]:
async function generateAnswer(input: {question: string, context: string}): Promise<string> {
    const messages = [
        { role: "system", content: `Answer the user's question based only on the content below:\n\n${input.context}` },
        { role: "user", content: input.question }
    ];

    const chatCompletion = await traceCreateCompletion({
        model: "gpt-3.5-turbo",
        messages: messages,
        temperature: 0,
    });
    return chatCompletion.choices[0].message.content;
}

In [8]:
const traceGenerateWikiSearch = traceable(generateWikiSearch);
const traceRetrieve = traceable(retrieve, { name: "Retrieve Wiki", run_type: "retriever" });
const traceGenerateAnswer = traceable(generateAnswer);

const traceRagPipeline = traceable(async ({ question }, numDocuments: number = 2) => {
    const query = await traceGenerateWikiSearch({ question });
    const retrieverResults = await traceRetrieve({query, numDocuments });
    const context = retrieverResults.map(result => result.page_content).join("\n\n");
    const answer = await traceGenerateAnswer({ question, context });
    return answer
},
{ name: "Wiki RAG Pipeline", run_type: "llm" });

In [9]:
await traceRagPipeline({question: "When was the Apple Vision Pro released in the US?"})

"The Apple Vision Pro was released in the United States on February 2, 2024."

![Screenshot of Trace View in LangSmith](trace_view.png)

## Run the pipeline on some test cases

Before deploying to an initial set of users, it's often helpful to create a test set of a few examples, then run your pipeline on the test set.

LangSmith makes it easy to run custom evaluations (both LLM and heuristic-based) to score the results.

In [13]:
// Create a dataset to be used for testing using the LangSmith client
const examples = [
    ["When was the Apple Vision Pro released in the US?", "The Apple Vision Pro was released in the United States on February 2, 2024."],
    ["What is LangChain?", "LangChain is an open-source framework for building applications using large language models."],
    ["Who is the chairman of OpenAI?", "Bret Taylor is the chairman of OpenAI"]
]

const datasetName = "Wikipedia RAG Pipeline";
const dataset = await langsmith.createDataset(datasetName);

await Promise.all(examples.map(async ([question, answer]) => {
    await langsmith.createExample(
        { question },
        { answer },
        { datasetId: dataset.id },
    );
}));

[ undefined, undefined, undefined ]

![Dataset](dataset.png)

In [14]:
// Run a set of tests on the dataset and compare them in LangSmith
// First, set up evaluators to run against the test results
import type { DynamicRunEvaluatorParams } from "langchain/smith";
import { runOnDataset, Criteria, LabeledCriteria } from "langchain/smith";
// An illustrative custom evaluator example
const containsOpenAI = async ({
    run,
    example,
    input,
    prediction,
    reference,
}: DynamicRunEvaluatorParams) => {
    return {
        key: "contains_openai",
        score: prediction.output.includes("OpenAI"),
    };
};

const evaluators: RunEvalConfig["evaluators"] = [
  // LangChain's built-in evaluators
  Criteria("conciseness"),
  LabeledCriteria("correctness"),
    
  // Custom evaluators can be user-defined RunEvaluator's
  // or a compatible function
  containsOpenAI,
];

In [29]:
// Use `runOnDataset` to run the pipeline against examples in the Dataset
await runOnDataset(
    traceRagPipeline, 
    datasetName,
    {
        evaluationConfig: {
            evaluators,
        },
        projectName: "test-2-documents",
    }
);

Predicting: ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓ 100.00% | 3/3

Completed
Running Evaluators: ▓▓▓▓▓▓▓▓▓▓▓▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░ 33.33% | 1/3

Running Evaluators: ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░░░░░░░░░░░ 66.67% | 2/3

Running Evaluators: ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓ 100.00% | 3/3



{
  projectName: "test-2-documents",
  results: {
    "60ec0fe3-f790-4475-ba50-9446327d11da": {
      execution_time: 1581,
      feedback: [
        {
          id: "a0d04766-5b04-4bc7-a918-0207cbd661e3",
          run_id: "9a72fb94-5f56-43fc-9dd4-cb9b99a42f2d",
          key: "contains_openai",
          score: false,
          value: undefined,
          correction: undefined,
          comment: undefined,
          feedback_source: [Object]
        },
        {
          id: "f94de4bb-3367-4971-87c2-8740b261c1c5",
          run_id: "9a72fb94-5f56-43fc-9dd4-cb9b99a42f2d",
          key: "conciseness",
          score: 1,
          value: "Y",
          correction: undefined,
          comment: "The criterion for this assessment is conciseness. To evaluate this, we need to consider whether the "... 495 more characters,
          feedback_source: [Object]
        },
        {
          id: "be60f10d-ff6e-4c46-80c9-07cedf9b8b61",
          run_id: "9a72fb94-5f56-43fc-9dd4-cb9b99a42f2d"

In [32]:
// Let's now execute a test of the pipeline where we retrieve 4 pages instead of 1

// Wrapper function with numDocuments set to 4
const traceRagPipelineFourDocuments = async ({ question }) => {
    return traceRagPipeline({ question }, 4);
};


await runOnDataset(
    traceRagPipelineFourDocuments,
    datasetName,
    {
        evaluationConfig: {
            evaluators,
        },
        projectName: "test-4-documents",
    }
);

Predicting: ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓ 100.00% | 3/3

Completed
Running Evaluators: ▓▓▓▓▓▓▓▓▓▓▓▓▓░░░░░░░░░░░░░░░░░░░░░░░░░░░ 33.33% | 1/3

Running Evaluators: ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓░░░░░░░░░░░░░ 66.67% | 2/3

Running Evaluators: ▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓ 100.00% | 3/3



{
  projectName: "test-4-documents",
  results: {
    "60ec0fe3-f790-4475-ba50-9446327d11da": {
      execution_time: 3015,
      feedback: [
        {
          id: "4be73ef6-54b7-4b01-a8b2-8121f9949f46",
          run_id: "535ed6c7-ba5f-438d-8a27-2faf42e9629e",
          key: "contains_openai",
          score: false,
          value: undefined,
          correction: undefined,
          comment: undefined,
          feedback_source: [Object]
        },
        {
          id: "6caacef1-d2bf-4247-8206-9e185e77b088",
          run_id: "535ed6c7-ba5f-438d-8a27-2faf42e9629e",
          key: "conciseness",
          score: 1,
          value: "Y",
          correction: undefined,
          comment: "The criterion for this assessment is conciseness. This means the submission should be brief, to the "... 636 more characters,
          feedback_source: [Object]
        },
        {
          id: "a7e258e6-c88b-44ca-8ba6-9677c09a1583",
          run_id: "535ed6c7-ba5f-438d-8a27-2faf42e9629e"

![Screenshot of Test Results in LangSmith](test_results.png)

![Screenshot of Comparison View in LangSmith](comparison_view.png)

## Tracing

Let's say you've deployed your application to production or to an initial set of users. 

You can view traces of your application in LangSmith and drill down by various attributes to get statistics on a specific set of traces.

You can also attach feedback to your runs (such as through a thumbs up/down button) [using the LangSmith client](https://docs.smith.langchain.com/tracing/faq/logging_feedback#collecting-feedback-programmatically), and filter on traces with a certain feedback score in the UI.

In [26]:
// Here we execute the pipeline on a number of potential user inputs, simulating how users might interact with it.
import { pLimit } from "https://deno.land/x/p_limit@v1.0.0/mod.ts";
const limit = pLimit(5); // Add a concurrency limit to avoid getting rate limited

async function runRagPipelines(questions: string[]): Promise<string[]> {
    const promises = questions.map(question =>
        limit(() => traceRagPipeline({ question }))
    );

    try {
        const results = await Promise.all(promises);
        return results;
    } catch (error) {
        console.error("Error running RAG Pipelines:", error);
        throw error;
    }
}

// Example usage
const questions: string[] = [
    "When was Sam Altman removed from OpenAI?",
    "What is the significance of the James Webb Space Telescope's first images?",
    "How did the international community respond to the crisis in Ukraine?",
    "What is the status of the COVID-19 vaccine distribution worldwide?",
    "What are the outcomes of the recent G7 summit?",
    "Who are the leading figures in climate change activism today?",
    "What is Vanta?",
    "How many majors has Jannik Sinner won?"
];
await runRagPipelines(questions)

[
  "Sam Altman was removed from OpenAI on November 17, 2023.",
  "The significance of the James Webb Space Telescope's first images, specifically the Webb's First Dee"... 395 more characters,
  "The international community responded to the crisis in Ukraine by imposing sanctions on Russia and C"... 450 more characters,
  "As of 3 January 2024, 13.53 billion COVID-19 vaccine doses have been administered worldwide. Approxi"... 381 more characters,
  "The content provided does not mention any specific outcomes of the recent G7 summit.",
  "The content provided does not mention any specific leading figures in climate change activism.",
  "Vanta can refer to multiple things. It can be a city in Finland called Vantaa, a graphics accelerato"... 398 more characters,
  "Jannik Sinner has won one major, specifically the Australian Open in 2024."
]

![Screenshot of Trace View in LangSmith](trace_project_view.png)